# Resume Specialist Using Reflection Technique

In [ ]:
! pip install -U --quiet  langchain langgraph fireworks-ai openai

In [15]:
import os
from getpass import getpass
os.environ['FIREWORKS_API_KEY'] = getpass('Enter your FIREWORKS API key: ')
os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

Enter your FIREWORKS API key:  ········
Enter your OpenAI API Key:  ········


In [17]:
from langchain_community.chat_models.fireworks import ChatFireworks
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a resume assistant tasked with writing excellent resumes."
            " Generate the best resume possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    model_kwargs={"max_tokens": 32768},
)
generate = prompt | llm

In [18]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a resume reviewer evaluating a resume submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [19]:
from typing import List, Sequence

async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


def should_continue(state: List[BaseMessage]):
    print("$$$$ state lenght $$$$$ :",len(state))
    if len(state) > 4:
        # End after 2 iterations
        return END
    return "reflect"


In [20]:
from langgraph.graph import END, MessageGraph
builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")

In [21]:
builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [22]:
from openai import OpenAI
client = OpenAI()

audio_file = open("docs/resume.m4a", "rb")
resume_transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file, 
  response_format="text"
)
print(resume_transcription)

My name is Joe Smith. I got my PhD degree in computer science from University of Toronto in 2017. After that, I worked for X company as a machine learning engineer. My task was to develop different machine learning models for X project. I also mentored some junior developers as well. I also deployed and monitored the models in production. After that, I joined Y company in 2022 as a senior language model researcher. My task was to conduct research on LLM models and how to fine tune and also augment them with some techniques. I also prototyped some use cases using the available large language tools. About my skills, I am proficient in Python and I also have a good experience working with deep learning frameworks such as PyTorch and TensorFlow. I also have a good experience using Panda and SQL for data manipulation and I am able to work with machine learning libraries such as Scikit-learn. If you want to reach me, my email is joe.smith at example.com



In [23]:
input = HumanMessage(content = f"""Write a resume based on the following information: \n
                                Resume : {resume_transcription} \n
                                """)


async for event in graph.astream(input):
    print("---")

---
$$$$ state lenght $$$$$ : 2
---
---
$$$$ state lenght $$$$$ : 4
---
---
$$$$ state lenght $$$$$ : 6
---


In [24]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()

================================ Human Message =================================

Write a resume based on the following information: 

                                Resume : My name is Joe Smith. I got my PhD degree in computer science from University of Toronto in 2017. After that, I worked for X company as a machine learning engineer. My task was to develop different machine learning models for X project. I also mentored some junior developers as well. I also deployed and monitored the models in production. After that, I joined Y company in 2022 as a senior language model researcher. My task was to conduct research on LLM models and how to fine tune and also augment them with some techniques. I also prototyped some use cases using the available large language tools. About my skills, I am proficient in Python and I also have a good experience working with deep learning frameworks such as PyTorch and TensorFlow. I also have a good experience using Panda and SQL for data manipulation 